# TÓM TẮT VĂN BẢN BẰNG TF-IDF VÀ TEXTRANK

Notebook này thực hiện tóm tắt văn bản sử dụng thuật toán TextRank dựa trên TF-IDF và đánh giá kết quả bằng ROUGE metrics.

## Các bước thực hiện:

1. Đọc file XML và lưu vào Word (input.docx)
2. Biểu diễn các câu bằng vector TF-IDF
3. Tính độ tương đồng cosine giữa các câu
4. Mô hình hóa văn bản dưới dạng đồ thị
5. Áp dụng thuật toán TextRank
6. Lấy 10% câu có điểm cao nhất
7. Lưu bản tóm tắt vào Word (output_summary.docx)
8. Đọc DUC_SUM reference và lưu vào Word (Test_DUC_SUM.docx)
9. So sánh và đánh giá bằng ROUGE

---


## 1. Setup và Import Libraries


In [1]:
# Import các thư viện cơ bản
import os
import re
import math
import numpy as np
from collections import defaultdict, Counter
import xml.etree.ElementTree as ET
import warnings
warnings.filterwarnings('ignore')

# Import thư viện cho Word processing
try:
    from docx import Document
    from docx.shared import Inches
    print("✓ python-docx đã được cài đặt")
except ImportError:
    print("Cần cài đặt python-docx: pip install python-docx")
    import subprocess
    subprocess.run(["pip", "install", "python-docx"])
    from docx import Document

# Cài đặt các thư viện cần thiết nếu chưa có
try:
    import numpy as np
    print("✓ numpy đã sẵn sàng")
except ImportError:
    print("Cần cài đặt numpy: pip install numpy")
    import subprocess
    subprocess.run(["pip", "install", "numpy"])
    import numpy as np

print("✓ Tất cả thư viện đã được import thành công!")
print("✓ Notebook sẵn sàng để chạy tóm tắt văn bản")

✓ python-docx đã được cài đặt
✓ numpy đã sẵn sàng
✓ Tất cả thư viện đã được import thành công!
✓ Notebook sẵn sàng để chạy tóm tắt văn bản


In [2]:
class TextSummarizerTFIDFTextRank:
    """
    Lớp tóm tắt văn bản sử dụng TF-IDF và TextRank
    Viết tay các công thức toán học để minh họa cách tính
    """
    
    def __init__(self, damping_factor=0.85, max_iterations=100, tolerance=1e-6):
        self.damping_factor = damping_factor
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        
        # Dữ liệu câu
        self.sentences = []
        self.sentence_vectors = []
        self.vocabulary = set()
        self.word_doc_count = defaultdict(int)
        
        # Ma trận
        self.tfidf_matrix = None
        self.cosine_matrix = None
        self.adjacency_matrix = None
        self.transition_matrix = None
        self.textrank_scores = None
        
        print("✓ TextSummarizerTFIDFTextRank đã được khởi tạo")
        print(f"  - Damping factor: {damping_factor}")
        print(f"  - Max iterations: {max_iterations}")
        print(f"  - Tolerance: {tolerance}")

# Khởi tạo summarizer
summarizer = TextSummarizerTFIDFTextRank()

# Cấu hình đường dẫn
BASE_PATH = "/Users/yoliephan/Library/CloudStorage/OneDrive-Personal/Tài liệu/MASTER 2025/Đợt 1/Xử lý Ngôn Ngữ Tự Nhiên/MY"
DUC_TEXT_PATH = os.path.join(BASE_PATH, "DUC_TEXT", "train")
DUC_SUM_PATH = os.path.join(BASE_PATH, "DUC_SUM")

print(f"✓ Đường dẫn dữ liệu: {DUC_TEXT_PATH}")
print(f"✓ Đường dẫn reference: {DUC_SUM_PATH}")

✓ TextSummarizerTFIDFTextRank đã được khởi tạo
  - Damping factor: 0.85
  - Max iterations: 100
  - Tolerance: 1e-06
✓ Đường dẫn dữ liệu: /Users/yoliephan/Library/CloudStorage/OneDrive-Personal/Tài liệu/MASTER 2025/Đợt 1/Xử lý Ngôn Ngữ Tự Nhiên/MY/DUC_TEXT/train
✓ Đường dẫn reference: /Users/yoliephan/Library/CloudStorage/OneDrive-Personal/Tài liệu/MASTER 2025/Đợt 1/Xử lý Ngôn Ngữ Tự Nhiên/MY/DUC_SUM


## 2. Bước 1: Đọc File XML và Xử Lý Văn Bản


In [3]:
def load_xml_document(self, file_path):
    """
    Đọc và xử lý file XML từ DUC dataset
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except Exception as e:
        print(f"Lỗi khi đọc file {file_path}: {e}")
        return ""

def preprocess_text(self, text):
    """
    Tiền xử lý văn bản:
    - Loại bỏ thẻ XML/HTML
    - Tách câu dựa trên dấu câu
    - Làm sạch văn bản
    """
    # Loại bỏ thẻ XML/HTML
    text = re.sub(r'<[^>]+>', '', text)
    
    # Tách câu dựa trên dấu chấm, chấm than, chấm hỏi
    sentences = re.split(r'[.!?]+', text)
    
    # Làm sạch từng câu
    cleaned_sentences = []
    for sentence in sentences:
        # Loại bỏ khoảng trắng thừa
        sentence = sentence.strip()
        if len(sentence) > 10:  # Chỉ giữ câu có ít nhất 10 ký tự
            # Chuyển về chữ thường cho việc xử lý
            sentence_clean = sentence.lower()
            # Loại bỏ ký tự đặc biệt nhưng giữ nguyên câu gốc để hiển thị
            cleaned_sentences.append({
                'original': sentence,
                'processed': re.sub(r'[^a-zA-Z0-9\s]', '', sentence_clean)
            })
    
    return cleaned_sentences

def save_to_word(self, content, filename):
    """
    Lưu nội dung vào file Word
    """
    doc = Document()
    doc.add_heading('Document Content', 0)
    
    if isinstance(content, list):
        for i, item in enumerate(content, 1):
            if isinstance(item, dict):
                doc.add_paragraph(f"{i}. {item['original']}")
            else:
                doc.add_paragraph(f"{i}. {item}")
    else:
        doc.add_paragraph(content)
    
    doc.save(filename)
    print(f"✓ Đã lưu vào file: {filename}")

# Thêm các phương thức vào class
TextSummarizerTFIDFTextRank.load_xml_document = load_xml_document
TextSummarizerTFIDFTextRank.preprocess_text = preprocess_text
TextSummarizerTFIDFTextRank.save_to_word = save_to_word

print("✓ Đã thêm các phương thức xử lý văn bản")

✓ Đã thêm các phương thức xử lý văn bản


In [4]:
# Hiển thị danh sách tất cả file có sẵn
print("DANH SÁCH FILE CÓ SẴN TRONG DUC_TEXT:")
print("="*50)

if os.path.exists(DUC_TEXT_PATH):
    all_files = [f for f in os.listdir(DUC_TEXT_PATH) if not f.startswith('.')]
    all_files.sort()
    
    print(f"Tổng cộng: {len(all_files)} file")
    print("\nCác file có sẵn:")
    
    # Hiển thị file theo dạng cột
    for i, filename in enumerate(all_files):
        if i % 5 == 0 and i > 0:
            print()
        print(f"{filename:<10}", end=" ")
    print("\n")
    
    print("Bạn có thể chọn bất kỳ file nào từ danh sách trên.")
    print("Ví dụ: d061j, d062j, d063j, ...")
else:
    print(f"Thư mục không tồn tại: {DUC_TEXT_PATH}")

DANH SÁCH FILE CÓ SẴN TRONG DUC_TEXT:
Tổng cộng: 50 file

Các file có sẵn:
d061j      d062j      d063j      d064j      d065j      
d066j      d067f      d068f      d069f      d070f      
d071f      d072f      d073b      d074b      d075b      
d076b      d077b      d078b      d079a      d080a      
d081a      d082a      d083a      d084a      d085d      
d086d      d087d      d089d      d090d      d091c      
d092c      d093c      d094c      d095c      d096c      
d097e      d098e      d099e      d100e      d101e      
d102e      d103g      d104g      d105g      d106g      
d107g      d108g      d109h      d110h      d111h      

Bạn có thể chọn bất kỳ file nào từ danh sách trên.
Ví dụ: d061j, d062j, d063j, ...


In [5]:
# Demo: Đọc file XML với tùy chọn chọn file
if os.path.exists(DUC_TEXT_PATH):
    files = [f for f in os.listdir(DUC_TEXT_PATH) if not f.startswith('.')]
    files.sort()  # Sắp xếp để dễ tìm
    
    if files:
        print(f"Có {len(files)} file có sẵn trong DUC_TEXT:")
        print("Một số file mẫu:", files[:10])  # Hiển thị 10 file đầu
        
        # Tùy chọn chọn file
        demo_file_input = input("\nNhập tên file muốn xử lý (ví dụ: d061j) hoặc nhấn Enter để dùng file đầu tiên: ").strip()
        
        if demo_file_input and demo_file_input in files:
            demo_file = demo_file_input
            print(f"Đã chọn file: {demo_file}")
        elif demo_file_input and demo_file_input not in files:
            print(f"File '{demo_file_input}' không tồn tại. Sử dụng file đầu tiên: {files[0]}")
            demo_file = files[0]
        else:
            demo_file = files[0]
            print(f"Sử dụng file mặc định: {demo_file}")
        
        file_path = os.path.join(DUC_TEXT_PATH, demo_file)
        
        print(f"Đang đọc file: {demo_file}")
        
        # Đọc nội dung
        content = summarizer.load_xml_document(file_path)
        print(f"✓ Đã đọc {len(content)} ký tự")
        
        # Tiền xử lý và tách câu
        sentences = summarizer.preprocess_text(content)
        print(f"Đã tách thành {len(sentences)} câu")
        
        # Lưu câu gốc vào summarizer
        summarizer.sentences = sentences
        
        # Hiển thị 3 câu đầu tiên
        print("\n3 câu đầu tiên sau khi xử lý:")
        for i, sent in enumerate(sentences[:3], 1):
            print(f"{i}. {sent['original']}")
        
        # Lưu vào file Word (input.docx)
        input_filename = os.path.join(BASE_PATH, "input.docx")
        summarizer.save_to_word(sentences, input_filename)
        
        print(f"\nBước 1 hoàn thành: Đã xử lý {len(sentences)} câu và lưu vào input.docx")
    else:
        print("Không tìm thấy file nào trong thư mục DUC_TEXT")
else:
    print(f"Thư mục không tồn tại: {DUC_TEXT_PATH}")

Có 50 file có sẵn trong DUC_TEXT:
Một số file mẫu: ['d061j', 'd062j', 'd063j', 'd064j', 'd065j', 'd066j', 'd067f', 'd068f', 'd069f', 'd070f']
Đã chọn file: d061j
Đang đọc file: d061j
✓ Đã đọc 32448 ký tự
Đã tách thành 201 câu

3 câu đầu tiên sau khi xử lý:
1. Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heavily populated south coast to prepare for high winds, heavy rains and high seas
2. The storm was approaching from the southeast with sustained winds of 75 mph gusting to 92 mph
3. ``There is no need for alarm,'' Civil Defense Director Eugenio Cabral said in a television alert shortly before midnight Saturday
✓ Đã lưu vào file: /Users/yoliephan/Library/CloudStorage/OneDrive-Personal/Tài liệu/MASTER 2025/Đợt 1/Xử lý Ngôn Ngữ Tự Nhiên/MY/input.docx

Bước 1 hoàn thành: Đã xử lý 201 câu và lưu vào input.docx
Đã chọn file: d061j
Đang đọc file: d061j
✓ Đã đọc 32448 ký tự
Đã tách thành 201 câu

3 câu đầu tiên sau khi xử lý:
1. Hurricane Gilb

## 3. Bước 2: Biểu Diễn Câu Bằng Vector TF-IDF

**Công thức TF-IDF:**

- **TF (Term Frequency)** = (số lần xuất hiện của từ trong câu) / (tổng số từ trong câu)
- **IDF (Inverse Document Frequency)** = log(tổng số câu / số câu chứa từ đó)
- **TF-IDF** = TF × IDF


In [6]:
def build_vocabulary(self):
    """
    Xây dựng từ vựng từ tất cả các câu
    """
    print("Đang xây dựng từ vựng...")
    
    # Tách từ từ tất cả các câu
    all_words = []
    for sentence in self.sentences:
        words = sentence['processed'].split()
        # Lọc từ có ít nhất 2 ký tự
        words = [word for word in words if len(word) >= 2]
        all_words.extend(words)
        
        # Cập nhật từ vựng unique
        self.vocabulary.update(words)
        
        # Đếm số câu chứa mỗi từ
        unique_words = set(words)
        for word in unique_words:
            self.word_doc_count[word] += 1
    
    print(f"Tổng từ vựng: {len(self.vocabulary)} từ")
    print(f"Tổng từ (có lặp): {len(all_words)} từ")
    
    return list(self.vocabulary)

def calculate_tf(self, word, sentence_words):
    """
    Tính Term Frequency (TF) 
    TF = số lần xuất hiện của từ / tổng số từ trong câu
    """
    word_count = sentence_words.count(word)
    total_words = len(sentence_words)
    tf = word_count / total_words if total_words > 0 else 0
    return tf

def calculate_idf(self, word):
    """
    Tính Inverse Document Frequency (IDF) 
    IDF = log(tổng số câu / số câu chứa từ)
    """
    total_sentences = len(self.sentences)
    sentences_with_word = self.word_doc_count[word]
    
    if sentences_with_word == 0:
        return 0
    
    idf = math.log(total_sentences / sentences_with_word)
    return idf

def calculate_tfidf(self, word, sentence_words):
    """
    Tính TF-IDF - viết tay công thức
    TF-IDF = TF × IDF
    """
    tf = self.calculate_tf(word, sentence_words)
    idf = self.calculate_idf(word)
    tfidf = tf * idf
    return tfidf

def build_tfidf_matrix(self):
    """
    Xây dựng ma trận TF-IDF cho tất cả các câu
    """
    print("Đang xây dựng ma trận TF-IDF...")
    
    # Xây dựng từ vựng
    vocab_list = self.build_vocabulary()
    vocab_size = len(vocab_list)
    
    # Khởi tạo ma trận TF-IDF
    self.tfidf_matrix = np.zeros((len(self.sentences), vocab_size))
    
    # Tạo mapping từ word → index
    word_to_index = {word: i for i, word in enumerate(vocab_list)}
    
    # Tính TF-IDF cho từng câu
    for sent_idx, sentence in enumerate(self.sentences):
        sentence_words = sentence['processed'].split()
        sentence_words = [word for word in sentence_words if len(word) >= 2]
        
        # Tính TF-IDF cho mỗi từ trong câu
        for word in set(sentence_words):  # Chỉ tính cho từ unique trong câu
            if word in word_to_index:
                word_idx = word_to_index[word]
                self.tfidf_matrix[sent_idx, word_idx] = self.calculate_tfidf(word, sentence_words)
    
    print(f"✓ Ma trận TF-IDF: {self.tfidf_matrix.shape} (câu × từ vựng)")
    return self.tfidf_matrix, vocab_list

# Thêm methods vào class
TextSummarizerTFIDFTextRank.build_vocabulary = build_vocabulary
TextSummarizerTFIDFTextRank.calculate_tf = calculate_tf
TextSummarizerTFIDFTextRank.calculate_idf = calculate_idf
TextSummarizerTFIDFTextRank.calculate_tfidf = calculate_tfidf
TextSummarizerTFIDFTextRank.build_tfidf_matrix = build_tfidf_matrix

print("Đã thêm các phương thức tính TF-IDF")

Đã thêm các phương thức tính TF-IDF


In [7]:
# Demo: Xây dựng ma trận TF-IDF
if summarizer.sentences:
    # Xây dựng ma trận TF-IDF
    tfidf_matrix, vocab_list = summarizer.build_tfidf_matrix()
    
    # Demo: Hiển thị cách tính TF-IDF cho một từ cụ thể
    print("\nDEMO: Minh họa cách tính TF-IDF")
    print("="*50)
    
    # Chọn câu đầu tiên và từ phổ biến
    demo_sentence = summarizer.sentences[0]
    demo_words = demo_sentence['processed'].split()
    demo_words = [word for word in demo_words if len(word) >= 3]
    
    if demo_words:
        demo_word = demo_words[0]  # Chọn từ đầu tiên
        
        print(f"Câu demo: '{demo_sentence['original'][:100]}...'")
        print(f"Từ demo: '{demo_word}'")
        
        # Tính từng bước
        tf = summarizer.calculate_tf(demo_word, demo_words)
        idf = summarizer.calculate_idf(demo_word)
        tfidf = summarizer.calculate_tfidf(demo_word, demo_words)
        
        print(f"\n1. TF (Term Frequency):")
        print(f"   Số lần xuất hiện: {demo_words.count(demo_word)}")
        print(f"   Tổng số từ: {len(demo_words)}")
        print(f"   TF = {demo_words.count(demo_word)} / {len(demo_words)} = {tf:.6f}")
        
        print(f"\n2. IDF (Inverse Document Frequency):")
        print(f"   Tổng số câu: {len(summarizer.sentences)}")
        print(f"   Số câu chứa '{demo_word}': {summarizer.word_doc_count[demo_word]}")
        print(f"   IDF = log({len(summarizer.sentences)} / {summarizer.word_doc_count[demo_word]}) = {idf:.6f}")
        
        print(f"\n3. TF-IDF:")
        print(f"   TF-IDF = {tf:.6f} × {idf:.6f} = {tfidf:.6f}")
    
    print(f"\nBước 2 hoàn thành: Đã tạo ma trận TF-IDF {tfidf_matrix.shape}")
else:
    print("Chưa có dữ liệu câu để xử lý")

Đang xây dựng ma trận TF-IDF...
Đang xây dựng từ vựng...
Tổng từ vựng: 1039 từ
Tổng từ (có lặp): 3597 từ
✓ Ma trận TF-IDF: (201, 1039) (câu × từ vựng)

DEMO: Minh họa cách tính TF-IDF
Câu demo: 'Hurricane Gilbert swept toward the Dominican Republic Sunday, and the Civil Defense alerted its heav...'
Từ demo: 'hurricane'

1. TF (Term Frequency):
   Số lần xuất hiện: 1
   Tổng số từ: 27
   TF = 1 / 27 = 0.037037

2. IDF (Inverse Document Frequency):
   Tổng số câu: 201
   Số câu chứa 'hurricane': 56
   IDF = log(201 / 56) = 1.277953

3. TF-IDF:
   TF-IDF = 0.037037 × 1.277953 = 0.047332

Bước 2 hoàn thành: Đã tạo ma trận TF-IDF (201, 1039)


## 4. Bước 3: Tính Độ Tương Đồng Cosine

**Công thức Cosine Similarity:**

- **cosine_similarity(A, B)** = (A · B) / (|A| × |B|)
- **A · B** = tích vô hướng của hai vector
- **|A|** = độ dài (magnitude) của vector A = √(Σ(ai²))
- **|B|** = độ dài (magnitude) của vector B = √(Σ(bi²))


In [ ]:
def calculate_cosine_similarity(self, vector1, vector2):
    """
    Tính cosine similarity giữa hai vector - viết tay công thức
    cosine_similarity = (A · B) / (|A| × |B|)
    """
    # Tính tích vô hướng (dot product)
    dot_product = 0
    for i in range(len(vector1)):
        dot_product += vector1[i] * vector2[i]
    
    # Tính độ dài của vector A
    magnitude_a = 0
    for val in vector1:
        magnitude_a += val * val
    magnitude_a = math.sqrt(magnitude_a)
    
    # Tính độ dài của vector B  
    magnitude_b = 0
    for val in vector2:
        magnitude_b += val * val
    magnitude_b = math.sqrt(magnitude_b)
    
    # Tránh chia cho 0
    if magnitude_a == 0 or magnitude_b == 0:
        return 0
    
    # Tính cosine similarity
    cosine_sim = dot_product / (magnitude_a * magnitude_b)
    return cosine_sim

def build_cosine_matrix(self):
    """
    Xây dựng ma trận cosine similarity giữa tất cả các câu
    """
    print("Đang xây dựng ma trận cosine similarity...")
    
    if self.tfidf_matrix is None:
        print("Chưa có ma trận TF-IDF. Hãy chạy build_tfidf_matrix() trước.")
        return None
    
    num_sentences = self.tfidf_matrix.shape[0]
    self.cosine_matrix = np.zeros((num_sentences, num_sentences))
    
    # Tính cosine similarity cho mỗi cặp câu
    for i in range(num_sentences):
        for j in range(num_sentences):
            if i == j:
                self.cosine_matrix[i, j] = 1.0  # Similarity với chính nó = 1
            else:
                similarity = self.calculate_cosine_similarity(
                    self.tfidf_matrix[i], 
                    self.tfidf_matrix[j]
                )
                self.cosine_matrix[i, j] = similarity
    
    print(f"✓ Ma trận cosine similarity: {self.cosine_matrix.shape}")
    return self.cosine_matrix

def demonstrate_cosine_calculation(self, sent1_idx=0, sent2_idx=1):
    """
    Minh họa chi tiết cách tính cosine similarity
    """
    if self.tfidf_matrix is None or len(self.sentences) < 2:
        print("Không đủ dữ liệu để minh họa")
        return
    
    print(f"\nDEMO: Minh họa cách tính Cosine Similarity")
    print("="*60)
    
    # Lấy hai vector
    vector1 = self.tfidf_matrix[sent1_idx]
    vector2 = self.tfidf_matrix[sent2_idx]
    
    print(f"Câu 1: '{self.sentences[sent1_idx]['original'][:80]}...'")
    print(f"Câu 2: '{self.sentences[sent2_idx]['original'][:80]}...'")
    
    # Tính từng bước
    dot_product = np.dot(vector1, vector2)
    magnitude_a = np.linalg.norm(vector1)
    magnitude_b = np.linalg.norm(vector2)
    
    print(f"\n1. Tích vô hướng (A · B):")
    print(f"   dot_product = Σ(ai × bi) = {dot_product:.6f}")
    
    print(f"\n2. Độ dài vector:")
    print(f"   |A| = √(Σ(ai²)) = {magnitude_a:.6f}")
    print(f"   |B| = √(Σ(bi²)) = {magnitude_b:.6f}")
    
    if magnitude_a > 0 and magnitude_b > 0:
        cosine_sim = dot_product / (magnitude_a * magnitude_b)
        print(f"\n3. Cosine Similarity:")
        print(f"   cosine_sim = {dot_product:.6f} / ({magnitude_a:.6f} × {magnitude_b:.6f})")
        print(f"   cosine_sim = {cosine_sim:.6f}")
        
        # Giải thích ý nghĩa
        if cosine_sim > 0.8:
            interpretation = "rất tương tự"
        elif cosine_sim > 0.5:
            interpretation = "tương tự"
        elif cosine_sim > 0.3:
            interpretation = "hơi tương tự"
        else:
            interpretation = "không tương tự"
        print(f"   → Hai câu {interpretation}")
    else:
        print("\nMột trong hai vector có độ dài = 0")

# Thêm methods vào class
TextSummarizerTFIDFTextRank.calculate_cosine_similarity = calculate_cosine_similarity
TextSummarizerTFIDFTextRank.build_cosine_matrix = build_cosine_matrix
TextSummarizerTFIDFTextRank.demonstrate_cosine_calculation = demonstrate_cosine_calculation

print("Đã thêm các phương thức tính Cosine Similarity")

✓ Đã thêm các phương thức tính Cosine Similarity


In [ ]:
# Demo: Xây dựng ma trận cosine similarity
if summarizer.tfidf_matrix is not None:
    # Xây dựng ma trận cosine
    cosine_matrix = summarizer.build_cosine_matrix()
    
    # Minh họa cách tính cosine similarity
    if len(summarizer.sentences) >= 2:
        summarizer.demonstrate_cosine_calculation(0, 1)
    
    # Hiển thị thống kê ma trận cosine
    print(f"\nThống kê ma trận Cosine Similarity:")
    print(f"   - Kích thước: {cosine_matrix.shape}")
    print(f"   - Giá trị trung bình: {np.mean(cosine_matrix):.4f}")
    print(f"   - Giá trị max (không tính đường chéo): {np.max(cosine_matrix - np.eye(len(cosine_matrix))):.4f}")
    print(f"   - Giá trị min: {np.min(cosine_matrix):.4f}")
    
    # Hiển thị ma trận 5x5 đầu tiên
    display_size = min(5, len(summarizer.sentences))
    print(f"\nMa trận Cosine Similarity ({display_size}x{display_size} đầu tiên):")
    print("      ", end="")
    for j in range(display_size):
        print(f"S{j:<8}", end="")
    print()
    
    for i in range(display_size):
        print(f"S{i:<4} ", end="")
        for j in range(display_size):
            print(f"{cosine_matrix[i,j]:<8.3f}", end="")
        print()
    
    print(f"\nBước 3 hoàn thành: Đã tạo ma trận cosine similarity")
else:
    print("Chưa có ma trận TF-IDF")

## 5. Bước 4: Mô Hình Hóa Đồ Thị

Mô hình hóa văn bản dưới dạng đồ thị:

- **Đỉnh (Vertices)**: Mỗi câu là một đỉnh
- **Cạnh (Edges)**: Trọng số cosine similarity giữa các câu
- **Ngưỡng (Threshold)**: Chỉ tạo cạnh khi similarity > threshold


In [ ]:
def create_adjacency_matrix(self, threshold=0.1):
    """
    Tạo ma trận kề từ ma trận cosine similarity
    Nếu similarity > threshold thì có cạnh (giá trị = 1)
    """
    print(f"Đang tạo đồ thị với ngưỡng similarity: {threshold}")
    
    if self.cosine_matrix is None:
        print("Chưa có ma trận cosine similarity")
        return None
    
    n = self.cosine_matrix.shape[0]
    self.adjacency_matrix = np.zeros((n, n))
    
    # Tạo cạnh dựa trên ngưỡng similarity
    edge_count = 0
    for i in range(n):
        for j in range(n):
            if i != j and self.cosine_matrix[i, j] > threshold:
                self.adjacency_matrix[i, j] = 1
                edge_count += 1
    
    print(f"Đã tạo {edge_count} cạnh trong đồ thị")
    print(f"Ma trận kề: {self.adjacency_matrix.shape}")
    
    return self.adjacency_matrix

def create_transition_matrix(self):
    """
    Tạo ma trận chuyển tiếp cho TextRank
    Mỗi hàng được chuẩn hóa sao cho tổng = 1
    """
    print("Đang tạo ma trận chuyển tiếp...")
    
    if self.adjacency_matrix is None:
        print("Chưa có ma trận kề")
        return None
    
    n = self.adjacency_matrix.shape[0]
    self.transition_matrix = np.copy(self.adjacency_matrix).astype(float)
    
    # Chuẩn hóa từng hàng
    for i in range(n):
        row_sum = np.sum(self.transition_matrix[i])
        if row_sum > 0:
            # Có cạnh ra → chuẩn hóa
            self.transition_matrix[i] = self.transition_matrix[i] / row_sum
        else:
            # Không có cạnh ra → phân bố đều cho tất cả đỉnh
            self.transition_matrix[i] = np.ones(n) / n
    
    print(f"Ma trận chuyển tiếp: {self.transition_matrix.shape}")
    
    # Kiểm tra tính chuẩn hóa
    row_sums = np.sum(self.transition_matrix, axis=1)
    print(f"Kiểm tra chuẩn hóa: tổng hàng = {row_sums[0]:.6f} (should be 1.0)")
    
    return self.transition_matrix

def visualize_graph_info(self):
    """
    Hiển thị thông tin về đồ thị
    """
    if self.adjacency_matrix is None:
        print("Chưa có ma trận kề")
        return
    
    n = len(self.sentences)
    total_edges = np.sum(self.adjacency_matrix)
    
    print(f"\nThông tin đồ thị:")
    print(f"   - Số đỉnh (câu): {n}")
    print(f"   - Số cạnh: {int(total_edges)}")
    print(f"   - Mật độ: {total_edges / (n * (n-1)):.4f}")
    
    # Hiển thị degree của từng đỉnh
    in_degrees = np.sum(self.adjacency_matrix, axis=0)  # Số cạnh vào
    out_degrees = np.sum(self.adjacency_matrix, axis=1)  # Số cạnh ra
    
    print(f"\nThống kê degree:")
    print(f"   - In-degree trung bình: {np.mean(in_degrees):.2f}")
    print(f"   - Out-degree trung bình: {np.mean(out_degrees):.2f}")
    print(f"   - Max in-degree: {int(np.max(in_degrees))}")
    print(f"   - Max out-degree: {int(np.max(out_degrees))}")
    
    # Hiển thị một vài câu có degree cao nhất
    high_degree_indices = np.argsort(in_degrees)[-3:][::-1]
    print(f"\nTop 3 câu có nhiều liên kết nhất:")
    for i, idx in enumerate(high_degree_indices, 1):
        sentence_preview = self.sentences[idx]['original'][:60]
        print(f"   {i}. Câu {idx}: degree={int(in_degrees[idx])} - '{sentence_preview}...'")

# Thêm methods vào class
TextSummarizerTFIDFTextRank.create_adjacency_matrix = create_adjacency_matrix
TextSummarizerTFIDFTextRank.create_transition_matrix = create_transition_matrix
TextSummarizerTFIDFTextRank.visualize_graph_info = visualize_graph_info

print("Đã thêm các phương thức mô hình hóa đồ thị")

In [ ]:
# Demo: Tạo đồ thị từ ma trận cosine similarity
if summarizer.cosine_matrix is not None:
    # Tạo ma trận kề với ngưỡng phù hợp
    threshold = 0.1  # Có thể điều chỉnh
    adjacency_matrix = summarizer.create_adjacency_matrix(threshold)
    
    # Tạo ma trận chuyển tiếp
    transition_matrix = summarizer.create_transition_matrix()
    
    # Hiển thị thông tin đồ thị
    summarizer.visualize_graph_info()
    
    # Hiển thị ma trận kề (5x5 đầu tiên)
    display_size = min(5, len(summarizer.sentences))
    print(f"\nMa trận kề ({display_size}x{display_size} đầu tiên):")
    print("      ", end="")
    for j in range(display_size):
        print(f"S{j:<4}", end="")
    print()
    
    for i in range(display_size):
        print(f"S{i:<4} ", end="")
        for j in range(display_size):
            print(f"{int(adjacency_matrix[i,j]):<4}", end="")
        print()
    
    print(f"\nBước 4 hoàn thành: Đã mô hình hóa văn bản thành đồ thị")
else:
    print("Chưa có ma trận cosine similarity")

## 6. Bước 5: Thuật Toán TextRank

**Công thức TextRank (dựa trên PageRank):**

**TR(Vi) = (1-d) + d × Σ(TR(Vj)/C(Vj))**

Trong đó:

- **TR(Vi)**: Điểm TextRank của câu Vi
- **d**: Damping factor (thường = 0.85)
- **Vj**: Các câu có liên kết đến Vi
- **C(Vj)**: Số liên kết ra từ câu Vj

**Phương pháp Power Iteration:**

- Khởi tạo: TR₀ = [1/N, 1/N, ..., 1/N]
- Lặp: TR\_{k+1} = (1-d)/N + d × M^T × TR_k
- Dừng khi: ||TR\_{k+1} - TR_k|| < tolerance


In [ ]:
def calculate_textrank(self):
    """
    Tính TextRank bằng Power Iteration 
    """
    print("Đang tính TextRank bằng Power Iteration...")
    
    if self.transition_matrix is None:
        print("Chưa có ma trận chuyển tiếp")
        return None
    
    n = self.transition_matrix.shape[0]
    
    # Khởi tạo vector TextRank (phân bố đều)
    textrank_vector = np.ones(n) / n
    print(f"Khởi tạo: TR₀ = [1/{n}, 1/{n}, ..., 1/{n}] = {1/n:.6f}")
    
    print(f"\nCông thức TextRank:")
    print(f"   TR_new = (1-d)/N + d × M^T × TR_old")
    print(f"   Với d = {self.damping_factor}, N = {n}")
    print(f"   Teleport term = (1-d)/N = {(1-self.damping_factor)/n:.6f}")
    
    # Lưu lịch sử hội tụ
    history = []
    
    print(f"\nQuá trình lặp:")
    print(f"{'Vòng':<6} {'Thay đổi':<15} {'Top 3 điểm':<25}")
    print("-" * 50)
    
    for iteration in range(self.max_iterations):
        # Lưu vector cũ
        old_textrank = np.copy(textrank_vector)
        
        # Áp dụng công thức TextRank
        # TR = (1-d)/N + d * M^T * TR_old
        teleport_term = (1 - self.damping_factor) / n
        random_walk_term = self.damping_factor * np.dot(self.transition_matrix.T, textrank_vector)
        textrank_vector = teleport_term + random_walk_term
        
        # Tính sự thay đổi (norm L2)
        change = 0
        for i in range(len(textrank_vector)):
            diff = textrank_vector[i] - old_textrank[i]
            change += diff * diff
        change = math.sqrt(change)
        
        # Lưu lịch sử
        history.append((iteration + 1, change, np.copy(textrank_vector)))
        
        # Hiển thị top 3 điểm cao nhất
        top_indices = np.argsort(textrank_vector)[-3:][::-1]
        top_scores = [f"{textrank_vector[i]:.4f}" for i in top_indices]
        top_info = ", ".join(top_scores)
        
        print(f"{iteration+1:<6} {change:<15.8f} {top_info:<25}")
        
        # Kiểm tra hội tụ
        if change < self.tolerance:
            print(f"\nHội tụ sau {iteration + 1} vòng lặp!")
            break
    else:
        print(f"\nĐạt giới hạn {self.max_iterations} vòng lặp")
    
    self.textrank_scores = textrank_vector
    
    return textrank_vector, history

def demonstrate_textrank_calculation(self, sentence_idx=0):
    """
    Minh họa chi tiết cách tính TextRank cho một câu cụ thể
    """
    if self.textrank_scores is None or self.adjacency_matrix is None:
        print("Chưa tính TextRank")
        return
    
    print(f"\nDEMO: Minh họa cách tính TextRank cho câu {sentence_idx}")
    print("="*70)
    
    sentence = self.sentences[sentence_idx]['original'][:80]
    print(f"Câu: '{sentence}...'")
    
    n = len(self.sentences)
    
    # Tìm các câu liên kết đến câu này
    incoming_links = []
    for i in range(n):
        if self.adjacency_matrix[i, sentence_idx] == 1:
            incoming_links.append(i)
    
    print(f"\n1. CÁC CÂU LIÊN KẾT ĐẾN:")
    if incoming_links:
        for link_idx in incoming_links:
            link_sentence = self.sentences[link_idx]['original'][:50]
            outgoing_count = np.sum(self.adjacency_matrix[link_idx])
            print(f"   - Câu {link_idx}: '{link_sentence}...' (có {int(outgoing_count)} liên kết ra)")
    else:
        print("   - Không có liên kết đến")
    
    print(f"\n2. CÔNG THỨC TEXTRANK:")
    print(f"   TR(S{sentence_idx}) = (1-d)/N + d × Σ(TR(Si)/C(Si))")
    
    # Tính từng thành phần
    teleport_term = (1 - self.damping_factor) / n
    print(f"\n3. TÍNH TOÁN CHI TIẾT:")
    print(f"   a) Teleport term = (1-d)/N = (1-{self.damping_factor})/{n} = {teleport_term:.6f}")
    
    link_contribution = 0
    print(f"   b) Link contribution:")
    if incoming_links:
        for link_idx in incoming_links:
            tr_link = self.textrank_scores[link_idx]
            outgoing_count = np.sum(self.adjacency_matrix[link_idx])
            contribution = tr_link / outgoing_count if outgoing_count > 0 else 0
            link_contribution += contribution
            print(f"      - Từ câu {link_idx}: TR = {tr_link:.6f}, "
                  f"C = {int(outgoing_count)}, "
                  f"Contribution = {tr_link:.6f}/{int(outgoing_count)} = {contribution:.6f}")
        
        print(f"      Tổng link contribution = {link_contribution:.6f}")
    else:
        print(f"      Không có liên kết → contribution = 0")
    
    final_textrank = teleport_term + self.damping_factor * link_contribution
    actual_textrank = self.textrank_scores[sentence_idx]
    
    print(f"\n4. KẾT QUẢ CUỐI CÙNG:")
    print(f"   TR(S{sentence_idx}) = {teleport_term:.6f} + {self.damping_factor} × {link_contribution:.6f}")
    print(f"   TR(S{sentence_idx}) = {teleport_term:.6f} + {self.damping_factor * link_contribution:.6f}")
    print(f"   TR(S{sentence_idx}) = {final_textrank:.6f}")
    print(f"   TextRank thực tế: {actual_textrank:.6f}")
    print(f"   Sai số: {abs(final_textrank - actual_textrank):.8f}")

# Thêm methods vào class
TextSummarizerTFIDFTextRank.calculate_textrank = calculate_textrank
TextSummarizerTFIDFTextRank.demonstrate_textrank_calculation = demonstrate_textrank_calculation

print("Đã thêm thuật toán TextRank")

In [ ]:
# Demo: Tính TextRank
if summarizer.transition_matrix is not None:
    # Tính TextRank
    textrank_scores, history = summarizer.calculate_textrank()
    
    # Minh họa cách tính cho một câu
    if len(summarizer.sentences) > 0:
        summarizer.demonstrate_textrank_calculation(0)
    
    # Hiển thị kết quả TextRank
    print(f"\nKẾT QUẢ TEXTRANK:")
    print("="*60)
    
    # Tạo danh sách kết quả
    results = []
    for i, score in enumerate(textrank_scores):
        results.append({
            'index': i,
            'score': score,
            'sentence': summarizer.sentences[i]['original']
        })
    
    # Sắp xếp theo điểm giảm dần
    results.sort(key=lambda x: x['score'], reverse=True)
    
    print(f"{'Hạng':<6} {'Điểm TR':<12} {'Câu':<60}")
    print("-" * 80)
    
    # Hiển thị top 10
    for rank, result in enumerate(results[:10], 1):
        sentence_preview = result['sentence'][:55]
        print(f"{rank:<6} {result['score']:<12.6f} {sentence_preview}...")
    
    # Thống kê
    print(f"\nThống kê TextRank:")
    print(f"   - Tổng điểm: {np.sum(textrank_scores):.6f}")
    print(f"   - Điểm trung bình: {np.mean(textrank_scores):.6f}")
    print(f"   - Độ lệch chuẩn: {np.std(textrank_scores):.6f}")
    print(f"   - Điểm cao nhất: {np.max(textrank_scores):.6f}")
    print(f"   - Điểm thấp nhất: {np.min(textrank_scores):.6f}")
    
    print(f"\nBước 5 hoàn thành: Đã tính TextRank cho {len(textrank_scores)} câu")
else:
    print("Chưa có ma trận chuyển tiếp")

## 7. Bước 6: Tạo Bản Tóm Tắt (Lấy 10% Câu Quan Trọng Nhất)


In [ ]:
def generate_summary(self, summary_ratio=0.1):
    """
    Tạo bản tóm tắt bằng cách chọn top câu có điểm TextRank cao nhất
    """
    print(f"Đang tạo bản tóm tắt với tỷ lệ {summary_ratio*100}%...")
    
    if self.textrank_scores is None:
        print("Chưa tính TextRank")
        return None
    
    total_sentences = len(self.sentences)
    num_summary_sentences = max(1, int(total_sentences * summary_ratio))
    
    print(f"Chọn {num_summary_sentences} câu từ {total_sentences} câu gốc")
    
    # Tạo danh sách câu với điểm số và vị trí gốc
    sentence_scores = []
    for i, score in enumerate(self.textrank_scores):
        sentence_scores.append({
            'index': i,
            'score': score,
            'sentence': self.sentences[i]['original']
        })
    
    # Sắp xếp theo điểm TextRank giảm dần
    sentence_scores.sort(key=lambda x: x['score'], reverse=True)
    
    # Chọn top câu
    selected_sentences = sentence_scores[:num_summary_sentences]
    
    # Sắp xếp lại theo thứ tự xuất hiện trong văn bản gốc
    selected_sentences.sort(key=lambda x: x['index'])
    
    # Tạo văn bản tóm tắt
    summary_text = []
    for item in selected_sentences:
        summary_text.append(item['sentence'])
    
    summary = {
        'sentences': selected_sentences,
        'text': ' '.join(summary_text),
        'ratio': summary_ratio,
        'original_count': total_sentences,
        'summary_count': num_summary_sentences
    }
    
    return summary

def display_summary(self, summary):
    """
    Hiển thị bản tóm tắt
    """
    if summary is None:
        print("Không có bản tóm tắt")
        return
    
    print(f"\nBẢN TÓM TẮT:")
    print("="*70)
    print(f"Tỷ lệ: {summary['ratio']*100}% ({summary['summary_count']}/{summary['original_count']} câu)")
    print(f"Độ dài: {len(summary['text'])} ký tự")
    print()
    
    # Hiển thị từng câu được chọn
    print("Các câu được chọn:")
    for i, item in enumerate(summary['sentences'], 1):
        print(f"{i}. [Câu {item['index']}, Điểm: {item['score']:.4f}]")
        print(f"   {item['sentence']}")
        print()
    
    print("Văn bản tóm tắt liên tục:")
    print("-" * 50)
    print(summary['text'])

# Thêm methods vào class
TextSummarizerTFIDFTextRank.generate_summary = generate_summary
TextSummarizerTFIDFTextRank.display_summary = display_summary

print("✓ Đã thêm các phương thức tạo tóm tắt")

In [ ]:
# Demo: Tạo bản tóm tắt và lưu vào Word
if summarizer.textrank_scores is not None:
    # Tạo tóm tắt với 10% câu
    summary = summarizer.generate_summary(summary_ratio=0.1)
    
    # Hiển thị tóm tắt
    summarizer.display_summary(summary)
    
    # Lưu tóm tắt vào file Word (output_summary.docx)
    if summary:
        output_filename = os.path.join(BASE_PATH, "output_summary.docx")
        
        # Tạo document Word với format đẹp
        doc = Document()
        doc.add_heading('BẢN TÓM TẮT VĂN BẢN BẰNG TEXTRANK', 0)
        
        # Thông tin tóm tắt
        doc.add_heading('Thông tin tóm tắt:', level=1)
        info_para = doc.add_paragraph()
        info_para.add_run(f"• Tỷ lệ tóm tắt: {summary['ratio']*100}%\n")
        info_para.add_run(f"• Số câu gốc: {summary['original_count']}\n")
        info_para.add_run(f"• Số câu tóm tắt: {summary['summary_count']}\n")
        info_para.add_run(f"• Độ dài: {len(summary['text'])} ký tự\n")
        
        # Văn bản tóm tắt
        doc.add_heading('Văn bản tóm tắt:', level=1)
        doc.add_paragraph(summary['text'])
        
        # Chi tiết các câu được chọn
        doc.add_heading('Chi tiết các câu được chọn:', level=1)
        for i, item in enumerate(summary['sentences'], 1):
            para = doc.add_paragraph()
            para.add_run(f"{i}. ").bold = True
            para.add_run(f"[Câu {item['index']}, Điểm TextRank: {item['score']:.4f}]\n")
            para.add_run(item['sentence'])
        
        doc.save(output_filename)
        print(f"\nĐã lưu bản tóm tắt vào: {output_filename}")
        
        print(f"\nBước 6-7 hoàn thành: Đã tạo và lưu bản tóm tắt")
    
    else:
        print("Không thể tạo tóm tắt")
        
else:
    print("Chưa tính TextRank")

## 8. Bước 8: Đọc DUC Reference Summary


In [ ]:
# Bước 8: Đọc DUC reference summary tương ứng
def load_duc_reference(doc_filename):
    """
    Đọc file DUC reference summary tương ứng với tài liệu
    """
    print(f"Đang tìm DUC reference cho: {doc_filename}")
    
    # Tìm file reference tương ứng
    reference_path = os.path.join(DUC_SUM_PATH, doc_filename)
    
    if os.path.exists(reference_path):
        try:
            with open(reference_path, 'r', encoding='utf-8') as f:
                reference_content = f.read()
            
            # Làm sạch nội dung reference
            reference_content = re.sub(r'<[^>]+>', '', reference_content)
            reference_content = reference_content.strip()
            
            print(f"✓ Đã đọc reference: {len(reference_content)} ký tự")
            print(f"✓ Preview: {reference_content[:100]}...")
            
            return reference_content
        except Exception as e:
            print(f"Lỗi khi đọc reference: {e}")
            return None
    else:
        print(f"Không tìm thấy file reference: {reference_path}")
        return None

# Demo: Đọc DUC reference và lưu vào Word
if os.path.exists(DUC_TEXT_PATH):
    files = [f for f in os.listdir(DUC_TEXT_PATH) if not f.startswith('.')]
    if files and 'demo_file' in locals():
        # Sử dụng cùng file đã chọn ở bước 1
        
        # Đọc reference
        reference_content = load_duc_reference(demo_file)
        
        if reference_content:
            # Lưu reference vào file Word (Test_DUC_SUM.docx)
            reference_filename = os.path.join(BASE_PATH, "Test_DUC_SUM.docx")
            
            doc = Document()
            doc.add_heading('DUC REFERENCE SUMMARY', 0)
            
            # Thông tin file
            doc.add_heading('Thông tin:', level=1)
            info_para = doc.add_paragraph()
            info_para.add_run(f"• File gốc: {demo_file}\n")
            info_para.add_run(f"• Đường dẫn reference: {os.path.join(DUC_SUM_PATH, demo_file)}\n")
            info_para.add_run(f"• Độ dài: {len(reference_content)} ký tự\n")
            
            # Nội dung reference
            doc.add_heading('Nội dung reference summary:', level=1)
            doc.add_paragraph(reference_content)
            
            doc.save(reference_filename)
            print(f"Đã lưu DUC reference vào: {reference_filename}")
            
            # Lưu vào biến để sử dụng trong evaluation
            duc_reference = reference_content
            
            print(f"\nBước 8 hoàn thành: Đã đọc và lưu DUC reference summary")
        else:
            duc_reference = None
            print("Không thể đọc DUC reference")
    else:
        print("Không có file để xử lý")
        duc_reference = None
else:
    print(f"Thư mục không tồn tại: {DUC_TEXT_PATH}")
    duc_reference = None

## 9. Bước 9: Đánh Giá Bằng ROUGE Metrics

**ROUGE (Recall-Oriented Understudy for Gisting Evaluation):**

- **ROUGE-1**: Đo overlap của unigrams (từ đơn)
- **ROUGE-2**: Đo overlap của bigrams (cặp từ)
- **ROUGE-L**: Đo Longest Common Subsequence (LCS)

**Công thức:**

- **Precision** = (số từ chung) / (số từ trong summary)
- **Recall** = (số từ chung) / (số từ trong reference)
- **F1-Score** = 2 × (Precision × Recall) / (Precision + Recall)


In [ ]:
def calculate_rouge_1(generated_summary, reference_summary):
    """
    Tính ROUGE-1 (unigram overlap) - viết tay công thức
    """
    # Tách từ và làm sạch
    gen_words = re.findall(r'\b\w+\b', generated_summary.lower())
    ref_words = re.findall(r'\b\w+\b', reference_summary.lower())
    
    # Đếm từ
    gen_word_count = Counter(gen_words)
    ref_word_count = Counter(ref_words)
    
    # Tính overlap
    overlap = 0
    for word in gen_word_count:
        if word in ref_word_count:
            overlap += min(gen_word_count[word], ref_word_count[word])
    
    # Tính precision, recall, f1
    precision = overlap / len(gen_words) if len(gen_words) > 0 else 0
    recall = overlap / len(ref_words) if len(ref_words) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'overlap': overlap,
        'gen_length': len(gen_words),
        'ref_length': len(ref_words)
    }

def calculate_rouge_2(generated_summary, reference_summary):
    """
    Tính ROUGE-2 (bigram overlap) - viết tay công thức
    """
    # Tách từ
    gen_words = re.findall(r'\b\w+\b', generated_summary.lower())
    ref_words = re.findall(r'\b\w+\b', reference_summary.lower())
    
    # Tạo bigrams
    gen_bigrams = []
    for i in range(len(gen_words) - 1):
        gen_bigrams.append((gen_words[i], gen_words[i+1]))
    
    ref_bigrams = []
    for i in range(len(ref_words) - 1):
        ref_bigrams.append((ref_words[i], ref_words[i+1]))
    
    # Đếm bigrams
    gen_bigram_count = Counter(gen_bigrams)
    ref_bigram_count = Counter(ref_bigrams)
    
    # Tính overlap
    overlap = 0
    for bigram in gen_bigram_count:
        if bigram in ref_bigram_count:
            overlap += min(gen_bigram_count[bigram], ref_bigram_count[bigram])
    
    # Tính precision, recall, f1
    precision = overlap / len(gen_bigrams) if len(gen_bigrams) > 0 else 0
    recall = overlap / len(ref_bigrams) if len(ref_bigrams) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'overlap': overlap,
        'gen_length': len(gen_bigrams),
        'ref_length': len(ref_bigrams)
    }

def calculate_lcs_length(seq1, seq2):
    """
    Tính độ dài Longest Common Subsequence - viết tay bằng dynamic programming
    """
    m, n = len(seq1), len(seq2)
    
    # Tạo bảng DP
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Fill bảng DP
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    
    return dp[m][n]

def calculate_rouge_l(generated_summary, reference_summary):
    """
    Tính ROUGE-L (LCS-based) - viết tay công thức
    """
    # Tách từ
    gen_words = re.findall(r'\b\w+\b', generated_summary.lower())
    ref_words = re.findall(r'\b\w+\b', reference_summary.lower())
    
    # Tính LCS
    lcs_length = calculate_lcs_length(gen_words, ref_words)
    
    # Tính precision, recall, f1
    precision = lcs_length / len(gen_words) if len(gen_words) > 0 else 0
    recall = lcs_length / len(ref_words) if len(ref_words) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'lcs_length': lcs_length,
        'gen_length': len(gen_words),
        'ref_length': len(ref_words)
    }

def evaluate_summary(generated_summary, reference_summary):
    """
    Đánh giá toàn diện bằng ROUGE metrics
    """
    print("Đang đánh giá bằng ROUGE metrics...")
    
    # Tính các ROUGE scores
    rouge_1 = calculate_rouge_1(generated_summary, reference_summary)
    rouge_2 = calculate_rouge_2(generated_summary, reference_summary)
    rouge_l = calculate_rouge_l(generated_summary, reference_summary)
    
    # Hiển thị kết quả
    print(f"\nKẾT QUẢ ĐÁNH GIÁ ROUGE:")
    print("="*60)
    
    print(f"\nROUGE-1 (Unigram Overlap):")
    print(f"   Precision: {rouge_1['precision']:.4f}")
    print(f"   Recall:    {rouge_1['recall']:.4f}")
    print(f"   F1-Score:  {rouge_1['f1']:.4f}")
    print(f"   Overlap:   {rouge_1['overlap']}/{rouge_1['gen_length']} vs {rouge_1['ref_length']} words")
    
    print(f"\nROUGE-2 (Bigram Overlap):")
    print(f"   Precision: {rouge_2['precision']:.4f}")
    print(f"   Recall:    {rouge_2['recall']:.4f}")
    print(f"   F1-Score:  {rouge_2['f1']:.4f}")
    print(f"   Overlap:   {rouge_2['overlap']}/{rouge_2['gen_length']} vs {rouge_2['ref_length']} bigrams")
    
    print(f"\nROUGE-L (LCS-based):")
    print(f"   Precision: {rouge_l['precision']:.4f}")
    print(f"   Recall:    {rouge_l['recall']:.4f}")
    print(f"   F1-Score:  {rouge_l['f1']:.4f}")
    print(f"   LCS:       {rouge_l['lcs_length']}/{rouge_l['gen_length']} vs {rouge_l['ref_length']} words")
    
    # Tính điểm tổng hợp
    overall_f1 = (rouge_1['f1'] + rouge_2['f1'] + rouge_l['f1']) / 3
    print(f"\nĐIỂM TỔNG HỢP:")
    print(f"   Overall F1-Score: {overall_f1:.4f}")
    
    # Đánh giá chất lượng
    if overall_f1 >= 0.5:
        quality = "Xuất sắc"
    elif overall_f1 >= 0.3:
        quality = "Tốt"
    elif overall_f1 >= 0.2:
        quality = "Khá"
    elif overall_f1 >= 0.1:
        quality = "Trung bình"
    else:
        quality = "Cần cải thiện"
    
    print(f"   Chất lượng tóm tắt: {quality}")
    
    return {
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'overall_f1': overall_f1,
        'quality': quality
    }

print("✓ Đã thêm các phương thức đánh giá ROUGE")

In [ ]:
# Demo: Đánh giá tóm tắt bằng ROUGE
if 'summary' in locals() and summary and 'duc_reference' in locals() and duc_reference:
    print("Bắt đầu đánh giá so sánh...")
    print("="*70)
    
    # Hiển thị hai văn bản cần so sánh
    print("Generated Summary:")
    print("-" * 30)
    print(summary['text'][:300] + "..." if len(summary['text']) > 300 else summary['text'])
    
    print(f"\nDUC Reference Summary:")
    print("-" * 30)
    print(duc_reference[:300] + "..." if len(duc_reference) > 300 else duc_reference)
    
    # Đánh giá ROUGE
    evaluation_results = evaluate_summary(summary['text'], duc_reference)
    
    # Demo: Minh họa cách tính ROUGE-1 chi tiết
    print(f"\nDEMO: Minh họa cách tính ROUGE-1")
    print("="*50)
    
    gen_words = re.findall(r'\b\w+\b', summary['text'].lower())
    ref_words = re.findall(r'\b\w+\b', duc_reference.lower())
    
    print(f"Generated words: {len(gen_words)} từ")
    print(f"Reference words: {len(ref_words)} từ")
    print(f"Từ chung (overlap): {evaluation_results['rouge_1']['overlap']} từ")
    
    print(f"\nCông thức ROUGE-1:")
    print(f"Precision = overlap / gen_length = {evaluation_results['rouge_1']['overlap']} / {len(gen_words)} = {evaluation_results['rouge_1']['precision']:.4f}")
    print(f"Recall = overlap / ref_length = {evaluation_results['rouge_1']['overlap']} / {len(ref_words)} = {evaluation_results['rouge_1']['recall']:.4f}")
    print(f"F1 = 2 × P × R / (P + R) = {evaluation_results['rouge_1']['f1']:.4f}")
    
    print(f"\nBước 9 hoàn thành: Đã đánh giá tóm tắt bằng ROUGE metrics")
    
elif 'summary' not in locals() or not summary:
    print("Chưa có bản tóm tắt để đánh giá")
elif 'duc_reference' not in locals() or not duc_reference:
    print("Chưa có DUC reference để so sánh")
else:
    print("Thiếu dữ liệu để đánh giá")